In [1]:
from dask.distributed import Client
client = Client(processes=False)
client

Client Scheduler: inproc://140.77.14.16/58525/1 Dashboard: http://localhost:8787,Cluster Workers: 1 Cores: 64 Memory: 270.47 GB


In [2]:
from bokeh.io import output_notebook
from bokeh.models import GeoJSONDataSource, HoverTool, CategoricalColorMapper, LinearColorMapper
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
output_notebook()

Loading BokehJS ...

In [3]:
import dask.dataframe as dd
import dask_geopandas as dg
import pandas as pd
from geopandas import GeoDataFrame
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm,numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import xlrd
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [4]:
df_insee=pd.read_csv('/datastore/complexnet/jlevyabi/geoloc/final.csv')
import re;pat = re.compile(r'''(-*\d+\.\d+ -*\d+\.\d+);*''');new_geo=[]
for geo in tqdm(df_insee.geometry_y):
    matches = pat.findall(geo)
    if matches:
        lst = Polygon([tuple(map(float, m.split())) for m in matches])
    else:
        lst=None
    new_geo.append(lst)

presentable_insee_df=df_insee.drop(["geometry","geometry_y","Unnamed: 0","idINSPIRE",
                                    "geometry_x","nbcar_x","id_x","id_y","idk"],axis=1)
presentable_insee_df=presentable_insee_df.drop("center",axis=1)
presentable_insee_df=presentable_insee_df.convert_objects(convert_numeric=True)
inter= presentable_insee_df.astype(float).round(3)
presentable_insee_df =inter.dropna(how='any')
val_idx = inter.index.isin(presentable_insee_df.index)
presentable_insee_df['geometry']=[new_geo[it] for it,x in enumerate(val_idx) if x]
geo_insee = GeoDataFrame(presentable_insee_df)
geom=geo_insee.geometry
geo_insee["owner_ratio"]=[round((row.men_prop+0.0)/row.ind_r,3) for it,row in tqdm(geo_insee.iterrows())]

100%|██████████| 2278213/2278213 [03:36<00:00, 10519.30it/s]
2277947it [13:21, 2841.93it/s]


In [5]:
geo_insee["center"]=[str(poly.centroid) for poly in tqdm(geo_insee.geometry)]

100%|██████████| 2277947/2277947 [07:26<00:00, 5098.49it/s]


In [6]:
geo_insee["owner_ratio"]=[round((row.men_prop+0.0)/row.ind_r,3) for it,row in tqdm(geo_insee.iterrows())]

2277947it [12:54, 2939.48it/s]


In [12]:
from bokeh.palettes import viridis, RdBu
from bokeh.models import ColorBar

geo_source = GeoJSONDataSource(geojson=geo_insee.head(1000).to_json())

color_mapper = LinearColorMapper(RdBu[11])

fig = figure(title='Fraction of rides that tip in cash')
fig.patches(xs='xs', ys='ys', alpha=0.9, source=geo_source, 
            color={'field': 'payment_type', 'transform': color_mapper}, 
            line_width=1, line_alpha=0.5, line_color='black')

hover = HoverTool(
    point_policy='follow_mouse',
    tooltips=('<div><b>Income</b>: @income</div>'
              '<div><b>Owner Ratio</b>: @owner_ratio</div>')
)
fig.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, 
                     location=(0, 0), 
                     label_standoff=12)
fig.add_layout(color_bar, 'right')

fig.xaxis.visible = False
fig.yaxis.visible = False
fig.grid.visible = True

#figures['cash-fraction'] = fig

show(fig)

In [8]:
from bokeh.resources import CDN
from bokeh.embed import file_html
html = file_html(fig, CDN, "/datastore/complexnet/jlevyabi/ml_soc_econ/data_files/UKSOC_rep/insee_income_or.html")

In [9]:
with open("/datastore/complexnet/jlevyabi/ml_soc_econ/data_files/UKSOC_rep/insee_income_or.html","w") as f:
    f.write(html)

In [1]:
import pandas as pd
linkedin_data=pd.read_csv("~/seacabo/data_files/salary_databases/linkedin_prof_jdn.csv",sep="\t",header=0)

In [2]:
linkedin_data.head()

,Unnamed: 0,id,summary,lkd,job_0,salary_0,job_1,salary_1,job_2,salary_2,job_3,salary_3,job_4,salary_4,job_5,salary_5,job_6,salary_6
0,5,100017919,press officer-eu parliament-i express only per...,"[('office', 2500.0)]",office,2500.000,NaN,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
1,13,100032180,multichannel/multicanal/open innovation/social...,"[('agent', 3038.625)]",agent,3038.625,NaN,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,14,1000352688,jpp // ragequit // graphic designer // overwat...,"[('designer', 2638.0), ('manager', 6708.333333...",designer,2638.000,manager,6708.333333,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,15,100036464,"co-fondatrice, directrice associee. conseil en...","[('conseil', 3059.75)]",conseil,3059.750,NaN,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,18,100057381,"artiste plasticienne, cours de peinture da...","[('artiste', 2075.0)]",artiste,2075.000,NaN,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [ ]:
l